## Assignment Week 3 - grk

#### Part 1: Using the TextBlob Sentiment Analyzer

In [13]:
# import modules

import pandas as pd
import re
from textblob import TextBlob

Import the movie review data as a data frame and ensure that the data is loaded properly.

In [14]:
# read the dataset as a dataframe
df = pd.read_csv("./assignments-data/labeledTrainData.tsv", sep="\t")

In [15]:
# view few rows to check dataset is loaded correcly
df.head()

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


How many of each positive and negative reviews are there?

In [16]:
# using value counts to get counts of postive and negetive sentiments
df["sentiment"].value_counts()

sentiment
1    12500
0    12500
Name: count, dtype: int64

Use TextBlob to classify each movie review as positive or negative. Assume that a polarity score greater than or equal to zero is a positive sentiment and less than 0 is a negative sentiment.

In [17]:
# this function converts input text to lower case and then performs sentiment analysis using TextBlob sentiment analyzer
# this function returns 1 (positive) if polarity of TextBlob sentiment analysis is greater than 0 otherwise returns 0 (negative)
def perform_sentiment_analysis(text):
    testimonial = TextBlob(text.lower())
    return 0 if testimonial.sentiment.polarity < 0 else 1

In [18]:
# perform sentiment analysis for each review and store in a new column
df["textblob_sentiment"] = df["review"].apply(perform_sentiment_analysis)

KeyboardInterrupt: 

In [ ]:
# view few rows
df.head()

Check the accuracy of this model. Is this model better than random guessing?

In [ ]:
# importing accuracy score module for calculating accuracy
from sklearn.metrics import accuracy_score

In [ ]:
# get accuracy score for sentiment generated by TextBlob against the labeled data
accuracy_score(df["sentiment"],df["textblob_sentiment"])

The sentiment analyzer model using `TextBlob` has an accuracy of $68.5$%. The original labeleled dataset has has $50$% positive and $50$% negetive sentiment. Hence, with random guessing we can expect to get a maximum of $50$% accuracy. However, the `TextBlob` sentiment analyzer model has accuracy higher than the $50$%, therefore we can say that the `TextBlob` model is better than _random guessing_.

For up to five points extra credit, use another prebuilt text sentiment analyzer, e.g., VADER, and repeat steps (3) and (4).

In [ ]:
# import VADER sentiment analyzer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [ ]:
# this function performs VADER sentiment analysis on the input text and returns 1 (poitive) 
# if the compound score from the analysis is greater than eqaul 0 otherwise 0 (negative)
def perform_sentiment_analysis_using_vader(text):
    analyzer = SentimentIntensityAnalyzer()
    sentiment_dict = analyzer.polarity_scores(text.lower())
    return 0 if sentiment_dict["compound"] < 0 else 1

In [ ]:
# perform VADER sentiment analysis for each review using above function and store in a new column
df["sentiment_with_vader"] = df["review"].apply(perform_sentiment_analysis_using_vader)

In [ ]:
# view few rows
df.head()

In [ ]:
# get accuracy score for sentiment generated by VADER against the labeled data
accuracy_score(df["sentiment"],df["sentiment_with_vader"])

We see that the accuracy of _Sentiment analysis_ using `VADER` sentiment analyzer did not change much from `TextBlob` sentiment analyzer, just $1$% increase in accuracy.

#### Part 2: Prepping Text for a Custom Model

In [ ]:
# import modules

import pandas as pd
import unicodedata
import sys
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# read the dataset into a dataframe
df = pd.read_csv("./datasets/labeledTrainData.tsv", sep="\t")

Convert all text to lowercase letters.

In [ ]:
# convert reviews to lower case and store in a new column
df["cleaned_review"] = df["review"].str.lower()

Remove punctuation and special characters from the text.

In [ ]:
# this function clean punctuation and symbols from input text
punctuation = dict.fromkeys(
[i for i in range(sys.maxunicode)
if unicodedata.category(chr(i)).startswith("P") or unicodedata.category(chr(i)).startswith("S")],
    None
 )
def remove_punctuation_symbols(text):
    return text.translate(punctuation)

In [ ]:
# remove punctuation and symbols from reviews using the above function 
df["cleaned_review"] = df["cleaned_review"].apply(remove_punctuation_symbols)

In [ ]:
# this function removes stopwords from input text
stop_words = stopwords.words("english")
def remove_stopwords(text):
    tokenized_words = word_tokenize(text)
    return " ".join([word for word in tokenized_words if word not in stop_words])

Remove stop words

In [ ]:
# remove stopwords from reviews using above function
df["cleaned_review"] = df["cleaned_review"].apply(remove_stopwords)

Apply NLTK’s PorterStemmer.

In [ ]:
# this function perform stemming on the input text using NLTK PorterStemmer
def porter_stemmer(text):
    tokenized_words = word_tokenize(text)
    porter = PorterStemmer()
    return " ".join([porter.stem(words) for words in tokenized_words])

In [ ]:
# apply stemming on reviews using above function
df["cleaned_review"] = df["cleaned_review"].apply(porter_stemmer)

In [ ]:
# view few rows after the above text preparation
df.head()

Create a bag-of-words matrix from your stemmed text (output from (4)), where each row is a word-count vector for a single movie review (see sections 5.3 & 6.8 in the Machine Learning with Python Cookbook). Display the dimensions of your bag-of-words matrix. The number of rows in this matrix should be the same as the number of rows in your original data frame.

In [ ]:
# Generate bag of words matrix using CountVectorizer
count_vec = CountVectorizer()
bag_of_words = count_vec.fit_transform(df["cleaned_review"])


In [ ]:
# dimensions of the generated bag-of-words matrix
bag_of_words.shape

In [ ]:
# dimensions of the dataframe
df.shape

we see that the number of rows of the bag-of-words matrix is same as the number of rows in the original dataframe i.e $25000$

Create a term frequency-inverse document frequency (tf-idf) matrix from your stemmed text, for your movie reviews (see section 6.9 in the Machine Learning with Python Cookbook). Display the dimensions of your tf-idf matrix. These dimensions should be the same as your bag-of-words matrix

In [ ]:
# generate term frequency-inverse document frequency (tf-idf) matrix using TfidfVectorizer
tfidf = TfidfVectorizer()
tfidf_feature_matrix = tfidf.fit_transform(df["cleaned_review"])

In [ ]:
# dimensions of te tf-idf feature matrix
tfidf_feature_matrix.shape

we see that the number of rows of the tf-idf matrix is same as the number of rows in the original dataframe i.e.$25000$

In [ ]:
df.head()

In [ ]:
df[(df["sentiment"] == 1) & (df["textblob_sentiment"] == 1)].shape

In [ ]:
df[(df["sentiment"] == 0) & (df["textblob_sentiment"] == 1)].shape